In [1]:
%matplotlib inline
from utils import *
from tqdm import *
from os.path import join
from matplotlib import pyplot as plt
from skimage.measure import block_reduce
from multiprocessing import Pool
import numpy as np
import librosa
from tqdm import tqdm

In [2]:
data_root = 'drumData'
n_fft = 1024
hop_length = n_fft/4
use_logamp = False # boost the brightness of quiet sounds
reduce_rows = 10 # how many frequency bands to average into one
reduce_cols = 1 # how many time steps to average into one
crop_rows = 32 # limit how many frequency bands to use
crop_cols = 32 # limit how many time steps to use
limit = None # set this to 100 to only process 100 samples

In [3]:
drumNames = ["kick", "tom", "snare", "clap", "hi.hat", "ride", "crash"]
drumFingerPrints = {}
drumSamples = {}
for d in drumNames:
    %time drumSamples[d] = np.load(join(data_root, d+'_samples.npy'))

CPU times: user 1.23 ms, sys: 202 ms, total: 204 ms
Wall time: 485 ms
CPU times: user 816 µs, sys: 14.9 ms, total: 15.7 ms
Wall time: 41.4 ms
CPU times: user 1.35 ms, sys: 97.1 ms, total: 98.5 ms
Wall time: 238 ms
CPU times: user 570 µs, sys: 46.5 ms, total: 47.1 ms
Wall time: 125 ms
CPU times: user 490 µs, sys: 5.8 ms, total: 6.29 ms
Wall time: 16.5 ms
CPU times: user 486 µs, sys: 8.67 ms, total: 9.16 ms
Wall time: 23 ms
CPU times: user 1.26 ms, sys: 28.1 ms, total: 29.4 ms
Wall time: 69.4 ms


In [11]:
'''
drum_features is a dictionary containing the drum type and 
the np.array of features. 
'''
drums_and_features = {} 
for drumName in drumNames: 
    file_path = './drumData/' + drumName + '_MIRfeatures.npy'
    drums_and_features[drumName] = np.load(file_path)
crash = drums_and_features['crash']
print(crash.shape)
print(crash[0].shape)

(723, 6, 24)
(6, 24)


In [15]:
drum_features = 

(7,)
